# 🔍 Spice Image Classification - Inference

This notebook demonstrates how to use the trained model for inference on new images.

## 📋 Steps:
1. Load trained model
2. Single image prediction
3. Batch prediction
4. Visualize predictions
5. Grad-CAM visualization

## 1️⃣ Import Libraries

In [1]:
import torch
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

from predict import SpicePredictor
from config import Config

# Enable inline plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print(f"PyTorch Version: {torch.__version__}")
print(f"Device: {Config.DEVICE}")

PyTorch Version: 2.10.0+cu128
Device: cuda


## 2️⃣ Set Paths

In [ ]:
# ============================================================
# SET YOUR PATHS HERE
# ============================================================

# Path to trained model
MODEL_PATH = "./outputs/best_model.pt"  # CHANGE THIS!

# Path to class names JSON
CLASS_NAMES_PATH = "./outputs/class_names.json"  # CHANGE THIS!

# Path to test image (for single prediction)
TEST_IMAGE_PATH = "./test_image.jpg"  # CHANGE THIS!

# Path to test images folder (for batch prediction)
TEST_FOLDER_PATH = "./test_images"  # CHANGE THIS!

# ============================================================

print(f"Model: {MODEL_PATH}")
print(f"Classes: {CLASS_NAMES_PATH}")
print(f"Test Image: {TEST_IMAGE_PATH}")
print(f"Test Folder: {TEST_FOLDER_PATH}")

## 3️⃣ Load Model

In [ ]:
# Create predictor
print("Loading model...")
predictor = SpicePredictor(
    model_path=MODEL_PATH,
    class_names_path=CLASS_NAMES_PATH
)

print(f"\n✓ Model loaded successfully!")
print(f"Number of classes: {predictor.num_classes}")
print(f"Classes: {predictor.class_names}")

## 4️⃣ Single Image Prediction

In [ ]:
# Predict single image
result = predictor.predict_image(TEST_IMAGE_PATH, top_k=5)

# Print results
predictor.predict_and_print(TEST_IMAGE_PATH, top_k=5)

In [ ]:
# Visualize prediction
predictor.visualize_prediction(
    TEST_IMAGE_PATH, 
    output_path='prediction_viz.png',
    top_k=5
)

# Display
from IPython.display import Image as IPImage
IPImage(filename='prediction_viz.png')

## 5️⃣ Access Prediction Details

In [ ]:
# Get prediction details
print("Prediction Details:")
print(f"Image: {result['image_path']}")
print(f"Predicted Class: {result['predicted_class']}")
print(f"Confidence: {result['confidence']*100:.2f}%")

print("\nTop-5 Predictions:")
for i, pred in enumerate(result['top_k_predictions'], 1):
    print(f"  {i}. {pred['class']:<20} - {pred['confidence_percent']:.2f}%")

## 6️⃣ Batch Prediction (Multiple Images)

In [ ]:
# Predict all images in folder
print(f"Predicting images in {TEST_FOLDER_PATH}...\n")

try:
    results = predictor.predict_folder(TEST_FOLDER_PATH, top_k=3)
    
    print(f"\n{'='*60}")
    print(f"Predicted {len(results)} images")
    print(f"{'='*60}")
    
except Exception as e:
    print(f"Error: {e}")
    print("Make sure TEST_FOLDER_PATH contains images!")

## 7️⃣ Visualize Multiple Predictions

In [ ]:
# Visualize multiple predictions in a grid
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

def visualize_predictions_grid(results, max_images=9):
    """Visualize predictions in a grid"""
    n_images = min(len(results), max_images)
    rows = int(n_images ** 0.5)
    cols = (n_images + rows - 1) // rows
    
    fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*4))
    if n_images == 1:
        axes = [axes]
    else:
        axes = axes.ravel()
    
    for idx, result in enumerate(results[:n_images]):
        # Load image
        img = Image.open(result['image_path'])
        
        # Display
        axes[idx].imshow(img)
        axes[idx].set_title(
            f"{result['predicted_class']}\n"
            f"Confidence: {result['confidence']*100:.1f}%",
            fontsize=10, fontweight='bold'
        )
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(n_images, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig('predictions_grid.png', dpi=150, bbox_inches='tight')
    plt.show()

# Visualize if we have results
try:
    if 'results' in locals() and len(results) > 0:
        visualize_predictions_grid(results, max_images=9)
except:
    print("Run batch prediction first!")

## 8️⃣ Grad-CAM Visualization (Advanced)

In [ ]:
# Grad-CAM visualization to see what the model focuses on
from gradcam import GradCAM, get_target_layer
from dataset_utils import get_transforms
from model import load_model_for_inference

# Load model for Grad-CAM
model = load_model_for_inference(MODEL_PATH, predictor.num_classes)

# Get target layer
target_layer = get_target_layer(model, Config.MODEL_NAME)

# Create Grad-CAM
gradcam = GradCAM(model, target_layer)

# Generate visualization
transform = get_transforms('test')
cam, pred_class, pred_prob = gradcam.visualize(
    TEST_IMAGE_PATH,
    transform,
    output_path='gradcam_output.png'
)

# Display
IPImage(filename='gradcam_output.png')

## 9️⃣ Interactive Prediction Widget

In [ ]:
# Interactive widget for file upload and prediction
from IPython.display import display
import ipywidgets as widgets

def predict_uploaded_image(change):
    """Predict when image is uploaded"""
    # Get uploaded file
    uploaded_file = list(change['new'].values())[0]
    content = uploaded_file['content']
    
    # Save temporarily
    temp_path = 'temp_upload.jpg'
    with open(temp_path, 'wb') as f:
        f.write(content)
    
    # Predict
    predictor.predict_and_print(temp_path, top_k=5)
    predictor.visualize_prediction(temp_path, 'upload_prediction.png', top_k=5)
    
    # Display
    display(IPImage(filename='upload_prediction.png'))

# Create upload widget
upload_widget = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)
upload_widget.observe(predict_uploaded_image, names='value')

print("Upload an image to predict:")
display(upload_widget)

## 🔟 Compare Predictions Across Different Images

In [ ]:
# Compare predictions for multiple images side-by-side
import pandas as pd

def create_comparison_table(results):
    """Create comparison table for predictions"""
    data = []
    for result in results:
        data.append({
            'Image': Path(result['image_path']).name,
            'Predicted Class': result['predicted_class'],
            'Confidence': f"{result['confidence']*100:.2f}%",
            'Top-2': result['top_k_predictions'][1]['class'] if len(result['top_k_predictions']) > 1 else 'N/A',
            'Top-2 Conf': f"{result['top_k_predictions'][1]['confidence_percent']:.2f}%" if len(result['top_k_predictions']) > 1 else 'N/A'
        })
    
    df = pd.DataFrame(data)
    return df

# Create comparison if we have batch results
try:
    if 'results' in locals() and len(results) > 0:
        comparison_df = create_comparison_table(results)
        print("\nPrediction Comparison:")
        display(comparison_df)
        
        # Save to CSV
        comparison_df.to_csv('prediction_comparison.csv', index=False)
        print("\n✓ Saved to prediction_comparison.csv")
except:
    print("Run batch prediction first!")

## ✅ Summary

You've successfully used the trained model for inference!

### What you can do:
- ✓ Predict single images
- ✓ Batch predict multiple images
- ✓ Visualize predictions with confidence scores
- ✓ Generate Grad-CAM visualizations
- ✓ Use interactive widget for uploads

### Files generated:
- `prediction_viz.png` - Single prediction visualization
- `predictions_grid.png` - Grid of multiple predictions
- `gradcam_output.png` - Grad-CAM visualization
- `prediction_comparison.csv` - Comparison table